# CH15. 콘텐스트 관리자와 else 블록
* with 문과 콘텍스트 관리자
* for, while, try 문에서 else 블록

## 15.1 이것 다음에 저것: if 문 이외에서의 else 블록
* for : for 루프가 완전히 실행된 후에 else 블록 실행 (중간에 break 에 걸리면 실행 안됨)
* while : 조건식이 거짓이 되어 while 루프를 빠져나온 후에 else 블록 실행 (중간에 break 에 걸리면 실행 안됨)
* try : try 블록에서 예외가 발생하지 않을 때만 else 블록 실행 (else 블록 내에서는 예외처리 안됨)

In [12]:
my_list = ['banana', 'candy']

for item in my_list:
    continue
else:
    raise ValueError('No banana found!')
        

ValueError: No banana found!

In [1]:
my_list = ['banana', 'candy']

for item in my_list:
    if item == 'banana':
        break
else:
    raise ValueError('No banana found!')
        

In [2]:
my_list = ['strawberry', 'candy']

for item in my_list:
    if item == 'banana':
        break
else:
    raise ValueError('No banana found!')
        

ValueError: No banana found!

In [ ]:
try:
    dangerous_call()
    after_call()
    # 가독성 측면에서 좋지 않음.
except OSError:
    log('OSError...')

In [ ]:
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()

* EAFP : Easier to Ask for Forgiveness than Permission
 - 올바르게 키나 속성이 있다고 가정하고 가정이 잘못되었을때 예외를 처리하는 코딩 스타일 (파이썬 스타일)

* LBYL : Look Before You Leap
 - 조회하기 전에 명시적으로 전제 조건을 검사, if 문을 많이 사용하는 특징
 - 다중 스레드 환경에서 '보는' 단계와 '뻗는' 단계가 경쟁 조건이 발생할 위험이 있음

## 15.2 콘텐스트 관리자와 with 블록

* 반복자가 for 문을 제어하기 위해 존재하는 것처럼, 콘텍스트 관리자 객체는 with 문을 제어하기 위해 존재
* with 문은 try/finally 패턴을 단순화하기 위해 설계 (finally 안에 중요한 리소스를 해제하거나 임시 상태를 복원하는 코드가 들어갈때)
* 콘텍스트 관리자 프로토콜은 `__enter__()` 와 `__exit__()` 메서드로 구성


In [5]:
try:
    fp_temp = open('mirror.py')
    src_temp = fp_temp.read(60)
finally:
    fp_temp.close()

In [1]:
with open('mirror.py') as fp:
    src = fp.read(60)

len(src)

60

In [2]:
fp

<_io.TextIOWrapper name='mirror.py' mode='r' encoding='UTF-8'>

In [3]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [4]:
fp.read(60)

ValueError: I/O operation on closed file.

In [6]:
class LookingGlass:

    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY' # as 뒤에 변수에 할당되는 객체
    
    def reverse_write(self, text):
        self.original_write(text[::-1]) # return?
        
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True
        

In [7]:
from mirror import LookingGlass

with LookingGlass() as what: # __enter__ 가 리턴하는 객체가 what 에 할당 (선택사항.)
    print('Alice, Kitty and Snowdrop')
    print(what)


pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [8]:
what

'JABBERWOCKY'

In [9]:
print('Back to normal.')

Back to normal.


In [10]:
from mirror import LookingGlass
manager = LookingGlass()
manager

In [11]:
monster = manager.__enter__()
monster == 'JABBERWOCKY'

True

In [12]:
monster

'JABBERWOCKY'

In [13]:
manager

In [14]:
manager.__exit__(None, None, None)

In [15]:
monster

'JABBERWOCKY'

In [17]:
## 안되는데.. 붙여서 해보면?
from mirror import LookingGlass
manager = LookingGlass()
monster = manager.__enter__()
print(monster == 'JABBERWOCKY')
print(monster)
manager.__exit__(None, None, None)
print(monster)
# 주피터 환경은 세션마다 설정이 새로 되는듯?

eurT
YKCOWREBBAJ
JABBERWOCKY


## 15.3 contextlib 유틸리티

* closing() : close() 메서드를 제공하지만 `__enter__() / __exit__()` 프로토콜을 구현하지 않는 객체로 부터 콘텍스트 관리자를 생성하는 함수
* suppress : 지정한 예외를 임시로 무시하는 콘텍스트 관리자
* @contextmanager : 클래스를 생성하고 프로토콜을 구현하는 대신, 간단한 제너레이터 함수로부터 콘텍스트 관리자를 생성할 수 있게 해주는 데커레이터
* ContextDecorator : 콘텍스트 관리자를 함수 데커레이터로도 사용할 수 있게 해주는 기반 클래스
* ExitStack : 여러 콘텍스트 관리자를 입력할 수 있게 해주는 콘텍스트 관리자, with 블록이 끝나면 `__exit__()` 메서드를 LIFO 순서로 호출

## 15.4 @contextmanager 사용하기
* 클래스 대신 yield 문 하나를 가진 제너레이터만 구현하면 됨

In [36]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write # with 절에서 호출
    yield 'JABBERWOCKY' # as 뒤에 변수에 할당
    sys.stdout.write = original_write # with 절이 끝나고 호출

In [37]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [38]:
what

'JABBERWOCKY'

In [39]:
# 위 코드가 문제가 될수 있는 상황
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    3/0
    print(what)

pordwonS dna yttiK ,ecilA


ZeroDivisionError: division by zero

In [41]:
print('back to normal')

lamron ot kcab


In [42]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    3/0
    print(what)

Alice, Kitty and Snowdrop


ZeroDivisionError: division by zero

In [43]:
# 예외처리까지 한 코드
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [44]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [45]:
what

'JABBERWOCKY'

`__exit__()` 메서드에서는 예외처리가 완료 되었음을 알려주기 위해 True 를 반환하고 반대로 예외를 전파하기 위해서는 None 을 반환하지만,

@contextmanager 에서는 예외를 모두 처리하고, 전파하기 위해서는 명시적으로 예외를 발생(raise)시켜야 한다.

In [46]:
from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                           encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

In [47]:
import csv

with inplace('test.csv', 'r', newline='') as (infh, outfh):
    reader = csv.reader(infh)
    writer = csv.writer(outfh)
    
    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)